In [2]:
import json
import csv
import pandas as pd
import numpy as np

In [3]:
def json2csv(json_name, csv_name, key):
    with open(json_name) as json_file:
        data = json.load(json_file)
    transaction_data = data[key]
    data_file = open(csv_name, 'w') 
    csv_writer = csv.writer(data_file)
    count = 0
    for tran in transaction_data:
        if count == 0: 
            # Writing headers of CSV file 
            header = tran.keys()
            csv_writer.writerow(header) 
            count += 1
        # Writing data of CSV file 
        csv_writer.writerow(tran.values())
    data_file.close() 

In [4]:
json2csv('allfidata_fish.json', 'bank_details.csv', 'data')

In [5]:
df = pd.read_csv('bank_details.csv')

In [6]:
df

,linkReferenceNumber,maskedAccountNumber,bank,Summary,Profile,Transactions
0,11fabe78-0a59-4c71-8fdb-57e54779d236,XXXXXXXXXX5055,FinShareBankServer,"{'currentBalance': '101666.33', 'currency': 'I...","{'Holders': {'type': 'SINGLE', 'Holder': [{'na...","{'startDate': '2019-07-11', 'endDate': '2019-0..."
1,4797daf8-75df-4359-8e61-a710d7e114fa,XXXXXXXXXX5055,FinShareBankServer,"{'branch': 'Jayanagar 4th Block', 'openingDate...","{'Holders': {'type': 'SINGLE', 'Holder': [{'na...","{'startDate': '2019-07-11', 'endDate': '2020-0..."
2,d68d3aa8-4d6d-4365-afd1-4f0822eb5d7e,XXXXXXXXXX0256,FinShareBankServer,"{'branch': 'JayaNagar 4th Block', 'openingDate...","{'Holders': {'type': 'SINGLE', 'Holder': [{'na...","{'startDate': '2019-07-11', 'endDate': '2020-0..."


In [7]:
df["Summary"][2]

"{'branch': 'JayaNagar 4th Block', 'openingDate': None, 'ifsc': 'ICIC0001124', 'accountType': 'RECURRING', 'maturityAmount': '61693.00', 'maturityDate': None, 'description': '12 Months Recurring Deposit', 'interestPayout': 'OnMaturity', 'interestRate': '5.15', 'principalAmount': '5000.00', 'tenureDays': '0', 'tenureMonths': '12', 'tenureYears': '0', 'recurringAmount': '5000.00', 'recurringDepositDay': '22', 'interestComputation': 'COMPOUND', 'compoundingFrequency': 'MONTHLY', 'interestPeriodicPayoutAmount': '0', 'interestOnMaturity': '1693', 'currentValue': '35000.00'}"

In [16]:
bank_name, account_type, account_no, principalAmount, currentValue, maturityAmount, recurringAmount, tenureDays, tenureMonths, tenureYears, interestRate, description  = [], [], [], [], [], [], [], [], [], [], [], []
for i in range(df.shape[0]):
    stri1 = df.at[i,"Summary"].replace("{","")
    stri1 = stri1.replace("'",'')
    #print(stri1)
    li = stri1.split(',')
    #print(li)
    type_ac = li[3].split(':')
    #print(type_ac)
    type_account1 = type_ac[1].strip() #Account type
    #print(type_account1)
    if type_account1 == "RECURRING":
        print("in")
        bank_name1 = df.at[i,"bank"] #Bank name
        account_no1 = df.at[i,"maskedAccountNumber"] #Account no
        maturity_am = li[4].split(":")
        maturity_amount1 = maturity_am[1].strip() #Maturity Amount
        desc = li[6].split(":")
        description1 = desc[1].strip() #Description
        int_rate = li[8].split(":")
        interest_rate1 = int_rate[1].strip() #Interest Rate
        principal_am = li[9].split(':')
        principal_amount1 = principal_am[1].strip() #Principal Amount
        tenure_da = li[10].split(":")
        tenure_days1 = tenure_da[1].strip() #Tenure Days
        tenure_mo = li[11].split(":")
        tenure_months1 = tenure_mo[1].strip() #Tenure Months
        tenure_ye = li[12].split(":")
        tenure_years1 = tenure_ye[1].strip() #Tenure Year
        rec_amount = li[13].split(":")
        recurring_amount1 = rec_amount[1].strip() #Recurring Amount
        cur_value = li[19].split(":")
        current_value1 = cur_value[1].strip() #Current Value
        bank_name.append(bank_name1)
        account_type.append(type_account1)
        account_no.append(account_no1)
        maturityAmount.append(maturity_amount1)
        description.append(description1)
        interestRate.append(interest_rate1)
        principalAmount.append(principal_amount1)
        tenureDays.append(tenure_days1)
        tenureMonths.append(tenure_months1)
        tenureYears.append(tenure_years1)
        recurringAmount.append(recurring_amount1)
        currentValue.append(current_value1)       
        

        print(bank_name, account_type, account_no, principalAmount, currentValue, maturityAmount, recurringAmount, tenureDays, tenureMonths, tenureYears, interestRate, description
    )
        #Account Type
        #print (bank_name1, current_balance1, account_no1, type_account1)


in
['FinShareBankServer'] ['RECURRING'] ['XXXXXXXXXX0256'] ['5000.00'] ['35000.00}'] ['61693.00'] ['5000.00'] ['0'] ['12'] ['0'] ['5.15'] ['12 Months Recurring Deposit']


In [17]:
df_out = pd.DataFrame({"bankName": bank_name, "accountNo": account_no, "typeAccount": account_type, 
                      "principalAmount": principalAmount, "maturityAmount": maturityAmount, "tenureDays": tenureDays, "tenureMonths" : tenureMonths,
                       "tenureYears": tenureYears, "currentValue": currentValue, "interestRate": interestRate, "description": description})
df_out

,bankName,accountNo,typeAccount,principalAmount,maturityAmount,tenureDays,tenureMonths,tenureYears,currentValue,interestRate,description
0,FinShareBankServer,XXXXXXXXXX0256,RECURRING,5000.00,61693.00,0,12,0,35000.00},5.15,12 Months Recurring Deposit


In [18]:
df_out.to_json("../web-app/server/src/rdDetails.json", orient = 'records')